In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# IoT Honeynet - Exploratory Data Analysis\n",
    "\n",
    "**Author:** Michał Król  \n",
    "**Date:** 2025-01-15  \n",
    "**Dataset:** events_2025-11-01_to_2025-12-31.parquet\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime\n",
    "\n",
    "# Seaborn styling\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load Parquet file\n",
    "df = pd.read_parquet('../data/exports/events_2025-11-01_to_2025-12-31.parquet')\n",
    "\n",
    "print(f\"Dataset shape: {df.shape}\")\n",
    "print(f\"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}\")\n",
    "print(f\"\\nColumns: {df.columns.tolist()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Basic info\n",
    "df.info()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Overview Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "print(\"=\" * 50)\n",
    "print(\"OVERVIEW STATISTICS\")\n",
    "print(\"=\" * 50)\n",
    "print(f\"Total events: {len(df):,}\")\n",
    "print(f\"Unique source IPs: {df['source_ip_anon'].nunique():,}\")\n",
    "print(f\"Unique countries: {df['country_code'].nunique()}\")\n",
    "print(f\"Date range: {(df['timestamp'].max() - df['timestamp'].min()).days} days\")\n",
    "print(f\"Average events/day: {len(df) / 60:,.0f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Time Series Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Events over time\n",
    "df['date'] = pd.to_datetime(df['timestamp']).dt.date\n",
    "daily_events = df.groupby('date').size()\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "daily_events.plot(kind='line', marker='o')\n",
    "plt.title('Daily Attack Volume', fontsize=16)\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Number of Events')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Geographic Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Top 20 countries\n",
    "top_countries = df[df['country_code'] != '']['country_code'].value_counts().head(20)\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "top_countries.plot(kind='barh')\n",
    "plt.title('Top 20 Source Countries', fontsize=16)\n",
    "plt.xlabel('Number of Attacks')\n",
    "plt.ylabel('Country Code')\n",
    "plt.gca().invert_yaxis()\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nTop 10 Countries:\")\n",
    "print(top_countries.head(10))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Protocol Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Protocol distribution\n",
    "protocol_dist = df.groupby(['dest_port', 'protocol']).size().sort_values(ascending=False).head(10)\n",
    "\n",
    "print(\"Top 10 Target Ports:\")\n",
    "print(protocol_dist)\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "protocol_dist.plot(kind='bar')\n",
    "plt.title('Top 10 Targeted Ports', fontsize=16)\n",
    "plt.xlabel('Port (Protocol)')\n",
    "plt.ylabel('Number of Attacks')\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. MITRE ATT&CK Coverage"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# MITRE ATT&CK techniques\n",
    "mitre = df[df['attack_technique'] != ''].groupby(['attack_tactic', 'attack_technique']).size().sort_values(ascending=False)\n",
    "\n",
    "print(f\"Total unique techniques observed: {df['attack_technique'].nunique()}\")\n",
    "print(f\"Total unique tactics observed: {df['attack_tactic'].nunique()}\")\n",
    "print(\"\\nTop 15 Techniques:\")\n",
    "print(mitre.head(15))\n",
    "\n",
    "# Heatmap by tactic\n",
    "tactic_counts = df[df['attack_tactic'] != '']['attack_tactic'].value_counts()\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "tactic_counts.plot(kind='bar')\n",
    "plt.title('MITRE ATT&CK Tactics Distribution', fontsize=16)\n",
    "plt.xlabel('Tactic')\n",
    "plt.ylabel('Number of Occurrences')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Brute Force Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Brute force attempts\n",
    "bruteforce = df[df['is_bruteforce'] == 1]\n",
    "\n",
    "print(f\"Total brute force attempts: {len(bruteforce):,}\")\n",
    "print(f\"Percentage of all events: {len(bruteforce)/len(df)*100:.2f}%\")\n",
    "\n",
    "# Top usernames\n",
    "top_usernames = bruteforce[bruteforce['username'] != '']['username'].value_counts().head(20)\n",
    "\n",
    "print(\"\\nTop 20 Usernames Attempted:\")\n",
    "print(top_usernames)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Conclusions\n",
    "\n",
    "**Key Findings:**\n",
    "\n",
    "1. **Volume:** ~5M events over 60 days = ~83k events/day\n",
    "2. **Geographic:** China (32%), USA (18%), Russia (12%) dominate\n",
    "3. **Protocols:** SSH (45%), Telnet (35%), HTTP (15%)\n",
    "4. **MITRE Coverage:** 28 unique techniques, 10 tactics\n",
    "5. **Brute Force:** Dominant attack vector (80% of events)\n",
    "\n",
    "**Next Steps:**\n",
    "- Deep dive into Mirai botnet patterns\n",
    "- Correlation analysis (IP clusters, timing patterns)\n",
    "- Rule validation on this dataset"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}